<a href="https://colab.research.google.com/github/thispath98/NLP/blob/main/Seq2Seq/Seq2Seq_with_Attention_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference
- [bentrevett](https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb)
- [나동빈](https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Sequence_to_Sequence_with_Attention_Tutorial.ipynb)


# Preparing Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [2]:
'''
영어와 독일어 토크나이저 불러오기
'''
!python -m spacy download en
!python -m spacy download de

     |████████████████████████████████| 12.0 MB 1.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9 MB 981 kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-py3-none-any.whl size=14907055 sha256=468f1c163ebcb763a43225a321dd5090fb3758f2d619d0323adc8ca28a92837e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ixm7_ixw/wheels/00/66/69/cb6c921610087d2cab339062345098e30a5ceb665360e7b32a
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.loa

In [3]:
spacy_en = spacy.load('en') # 영어 토크나이저
spacy_de = spacy.load('de') # 독일어 토크나이저

In [4]:
'''
토큰화가 잘 되는지 확인
'''
tokenized = spacy_en.tokenizer("I am an undergraduate student.")

for i, token in enumerate(tokenized):
    print(f'인덱스 {i}: {token.text}')

인덱스 0: I
인덱스 1: am
인덱스 2: an
인덱스 3: undergraduate
인덱스 4: student
인덱스 5: .


In [5]:
'''
독일어, 영어 토큰화 함수 정의
'''
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

In [6]:
'''
Field 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시한다.
- SRC(source): 독일어
- TRG(target): 영어
'''

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True)

In [7]:
'''
대표적인 영어-독일어 번역 데이터셋인 Multi30k를 불러온다.
'''

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

downloading training.tar.gz


100%|██████████| 1.21M/1.21M [00:04<00:00, 284kB/s]


downloading validation.tar.gz


100%|██████████| 46.3k/46.3k [00:00<00:00, 92.8kB/s]


downloading mmt_task1_test2016.tar.gz


100%|██████████| 66.2k/66.2k [00:00<00:00, 88.3kB/s]


In [8]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [9]:
'''
학습 데이터 중 하나 출력
'''
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [10]:
'''
Field 객체의 build_vocab 메서드를 이용해 영어와 독일어의 단어 사전 생성.
최소 2번 이상 등장한 단어만을 선택한다.
'''
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7855
len(TRG): 5893


In [11]:
'''
OOV는 0, 패딩은 1, <sos>, <eos>는 각각 2, 3이며 해당하는 단어에 대해 인덱스를 출력한다.
'''
print(TRG.vocab.stoi["OutOfVocabulary"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


In [12]:
'''
디바이스를 정의한다.
'''
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
'''
한 문장에 포함된 단어가 연속적으로 RNN에 입력되어야 하므로,
하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만든다.

이를 위해 BucketIterator를 사용한다.
매 epoch마다 문장을 새로 섞어 배치를 구성하고, 필요한 padding의 크기를 최소화할 수 있다.
배치 크기(batch size): 128
'''
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device=device
)

In [14]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"{i}번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[0]):
        print(f"{src[i][0].item()}")

    break

0번째 배치 크기: torch.Size([27, 128])
2
5
70
26
7
6
3008
128
7092
17
1499
4
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1


# Building the Seq2Seq Model


## Encoder
input sequence를 context vector로 인코딩한다.

hyperparemeter
- input_dim: 하나의 단어에 대한 원 핫 인코딩 차원
- embed_dim: 임베딩 차원
- enc_hidden_dim: encoder hidden state dimension
- dec_hidden_dim: decoder hidden state dimension
- dropout_ratio: 드랍아웃 비율

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, enc_hidden_dim, dec_hidden_dim, dropout_ratio):
        super().__init__()

        # embedding은 원 핫 인코딩을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # 양방향 GRU 레이어
        self.rnn = nn.GRU(embed_dim, enc_hidden_dim, bidirectional=True)

        # FC 레이어
        self.fc = nn.Linear(enc_hidden_dim * 2, dec_hidden_dim)

        # 드랍아웃
        self.dropout = nn.Dropout(dropout_ratio)
    
    # 인코더는 소스 문장을 입력으로 받아 문맥 벡터(context vector)를 반환   
    def forward(self, src):
        # src: [src len(단어 개수), batch size(배치 크기)]: 각 단어의 인덱스 정보
        embedded = self.dropout(self.embedding(src))
        # embedded = [src len(단어 개수), batch size(배치 크기), emb dim(임베딩 차원)]

        outputs, hidden = self.rnn(embedded)
        # outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * drection 수]: 전체 단어의 출력 정보
        # hidden: [레이어 개수 * direction 수, 배치 크기, 인코더 히든 차원]: 현재까지의 모든 단어의 정보

        # hidden은 [forward_1, backward_1, forward_2, backward_2, ...] 형태로 구성
        # 따라서 hidden[-2,:,:]은 forwards의 마지막 값
        # 따라서 hidden[-1,:,:]은 backwards의 마지막 값
        # 디코더의 첫번째 hidden (context) vector는 인코더의 마지막 hidden을 이용
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))

        # outputs은 Attention 목적으로 hidden은 context vector 목적으로 사용
        return outputs, hidden

## Attention
하나의 Attention은 인코더 전체 토큰에 대한 출력을 입력으로 받는 FC의 파라미터를 공유하여 사용한다.

(전체 인코더 출력(context vector) + 현재 디코더의 히든(input of decoder)) -> 디코더의 히든 -> 실제 Attention 값

hyperparameter
- enc_hidden_dim: encoder hidden state dimension
- dec_hidden_dim: decoder hidden state dimension

In [16]:
class Attention(nn.Module):
    def __init__(self, enc_hidden_dim, dec_hidden_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hidden_dim * 2) + dec_hidden_dim, dec_hidden_dim)
        self.v = nn.Linear(dec_hidden_dim, 1, bias=False)

    def forward(self, hidden, enc_outputs):
        # hidden: [배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # enc_outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
        batch_size = enc_outputs.shape[1]
        src_len = enc_outputs.shape[0]

        # 현재 디코더의 히든 상태(hidden state)를 src_len만큼 반복
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        enc_outputs = enc_outputs.permute(1, 0, 2)
        # hidden: [배치 크기, 단어 개수, 디코더 히든 차원]: 현재까지의 모든 단어의 정보
        # enc_outputs: [배치 크기, 단어 개수, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보

        energy = torch.tanh(self.attn(torch.cat((hidden, enc_outputs), dim=2)))
        # energy: [배치 크기, 단어 개수, 디코더 히든 차원]

        attention = self.v(energy).squeeze(2)
        # attention: [배치 크기, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들

        return F.softmax(attention, dim=1)

## Decoder
주어진 context vector를 타겟 문장으로 decoding한다.

decoding하는 과정에서 매번 인코더의 모든 출력에 대하여 attention을 수행한다.

hyperparameter
- output_dim: 하나의 단어에 대한 원 핫 인코딩 차원
- embed_dim: embedding dimension
- enc_hidden_dim: encoder hidden state dimension
- dec_hidden_dim: decoder hidden state dimension
- dropout_ratio: 드랍아웃 비율

In [17]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, enc_hidden_dim, dec_hidden_dim, dropout_ratio, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention

        # 임베딩(embedding)은 원 핫 인코딩 말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # GRU 레이어
        self.rnn = nn.GRU((enc_hidden_dim * 2) + embed_dim, dec_hidden_dim)

        # FC 레이어
        self.fc_out = nn.Linear((enc_hidden_dim * 2) + dec_hidden_dim + embed_dim, output_dim)

        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환
    def forward(self, input, hidden, enc_outputs):
        # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
        # hidden: [배치 크기, 히든 차원]
        # enc_outputs: [단어 개수, 배치 크기, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보
        input = input.unsqueeze(0)
        # input: [단어 개수 = 1, 배치 크기]

        embedded = self.dropout(self.embedding(input))
        # embedded: [단어 개수 = 1, 배치 크기, 임베딩 차원]

        attention = self.attention(hidden, enc_outputs)
        # attention: [배치 크기, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들
        attention = attention.unsqueeze(1)
        # attention: [배치 크기, 1, 단어 개수]: 실제 각 단어에 대한 어텐선(attention) 값들

        enc_outputs = enc_outputs.permute(1, 0, 2)
        # enc_outputs: [배치 크기, 단어 개수, 인코더 히든 차원 * 방향의 수]: 전체 단어의 출력 정보

        weighted = torch.bmm(attention, enc_outputs) # 행렬 곱 함수
        # weighted: [배치 크기, 1, 인코더 히든 차원 * 방향의 수]

        weighted = weighted.permute(1, 0, 2)
        # weighted: [1, 배치 크기, 인코더 히든 차원 * 방향의 수]

        rnn_input = torch.cat((embedded, weighted), dim=2)
        # rnn_input: [1, 배치 크기, 인코더 히든 차원 * 방향의 수 + embed_dim]: 어텐션이 적용된 현재 단어 입력 정보

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        # output: [단어 개수, 배치 크기, 디코더 히든 차원 * 방향의 수]
        # hidden: [레이어 개수 * 방향의 수, 배치 크기, 디코더 히든 차원]: 현재까지의 모든 단어의 정보

        # 현재 예제에서는 단어 개수, 레이어 개수, 방향의 수 모두 1의 값을 가짐
        # 따라서 output: [1, 배치 크기, 디코더 히든 차원], hidden: [1, 배치 크기, 디코더 히든 차원]
        # 다시 말해 output과 hidden의 값 또한 동일
        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        # prediction = [배치 크기, 출력 차원]

        # (현재 출력 단어, 현재까지의 모든 단어의 정보)
        return prediction, hidden.squeeze(0)

## Seq2Seq
위에서 정의한 인코더와 디코더를 가지고 있는 하나의 아키텍처이다.
- Encoder: 주어진 input sequence를 context vector로 인코딩한다.
- 인코더는 최종 hidden state와 모든 hidden state를 반환한다. 이때, 최종 hidden state는 디코더의 첫번째 hidden state로 사용된다.
- Decoder: 주어진 context vector를 target sequence로 디코딩한다.
- 디코더는 한 단어씩 넣어서 한 번씩 결과를 구한다.
- 디코더는 context vector 예측과 인코더의 모든 출력을 참고하여 attention을 진행한다.

Teacher forcing: 디코더의 prediction을 다음 입력으로 사용하지 않고, ground-truth를 다음 입력으로 사용하는 기법.


In [18]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]

        # 먼저 인코더를 거쳐 전체 출력과 문맥 벡터(context vector)를 추출
        enc_outputs, hidden = self.encoder(src)

        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원

        # 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫 번째 입력은 항상 <sos> 토큰
        input = trg[0, :]

        # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            # insert input sequence, 가장 최신의 hidden state 와 모든 인코더 hidden state를 디코더에 전달한다.
            # output: 예측한 결과 텐서(predictions)
            # hidden: 디코더 hidden state
            output, hidden = self.decoder(input, hidden, enc_outputs)

            # FC를 거쳐서 나온 현재의 출력 단어 정보
            # 매 반복마다 outputs에 예측된 결과를 저장한다.
            outputs[t] = output

            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            # 기본적으로 모든 예측의 50%를 teacher force한다.
            teacher_force = random.random() < teacher_forcing_ratio

            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기

        return outputs

# 학습(Training)
하이퍼 파라미터 설정 및 모델 초기화

In [19]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
ENCODER_HIDDEN_DIM = 512
DECODER_HIDDEN_DIM = 512
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [20]:
# 어텐션(attention) 객체 선언
attn = Attention(ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM)

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, DEC_DROPOUT_RATIO, attn)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

모델 가중치 파라미터 초기화

In [21]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

학습 및 평가 함수 정의

In [22]:
# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [23]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0
    
    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()

        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기, 출력 차원]
        output_dim = output.shape[-1]
        
        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어의 개수 - 1) * batch size, output dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어의 개수 - 1) * batch size]
        
        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산
        
        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # 파라미터 업데이트
        optimizer.step()
        
        # 전체 손실 값 계산
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [24]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0
    
    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]
            
            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].view(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

학습(training) 및 검증(validation) 진행
- Epoch는 10

In [25]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq_with_attention.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 2m 45s
	Train Loss: 5.039 | Train PPL: 154.383
	Validation Loss: 5.088 | Validation PPL: 162.090
Epoch: 02 | Time: 2m 44s
	Train Loss: 4.180 | Train PPL: 65.356
	Validation Loss: 4.486 | Validation PPL: 88.786
Epoch: 03 | Time: 2m 44s
	Train Loss: 3.522 | Train PPL: 33.857
	Validation Loss: 3.710 | Validation PPL: 40.862
Epoch: 04 | Time: 2m 44s
	Train Loss: 2.961 | Train PPL: 19.316
	Validation Loss: 3.415 | Validation PPL: 30.432
Epoch: 05 | Time: 2m 46s
	Train Loss: 2.567 | Train PPL: 13.033
	Validation Loss: 3.293 | Validation PPL: 26.932
Epoch: 06 | Time: 2m 43s
	Train Loss: 2.257 | Train PPL: 9.558
	Validation Loss: 3.163 | Validation PPL: 23.650
Epoch: 07 | Time: 2m 42s
	Train Loss: 1.989 | Train PPL: 7.310
	Validation Loss: 3.182 | Validation PPL: 24.091
Epoch: 08 | Time: 2m 43s
	Train Loss: 1.808 | Train PPL: 6.099
	Validation Loss: 3.221 | Validation PPL: 25.056
Epoch: 09 | Time: 2m 43s
	Train Loss: 1.638 | Train PPL: 5.145
	Validation Loss: 3.274 | Validati

In [27]:
# 학습된 모델 저장
from google.colab import files

files.download('seq2seq_with_attention.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

모델 최종 테스트(testing) 결과 확인

In [28]:
model.load_state_dict(torch.load('seq2seq_with_attention.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 3.168 | Test PPL: 23.770


모델 사용해보기

In [29]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        enc_outputs, hidden = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, enc_outputs)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [32]:
example_idx = 15

src = vars(test_dataset.examples[example_idx])['src']
trg = vars(test_dataset.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['ein', 'mädchen', 'in', 'einem', 'jeanskleid', 'läuft', 'über', 'einen', 'erhöhten', 'schwebebalken', '.']
타겟 문장: ['a', 'girl', 'in', 'a', 'jean', 'dress', 'is', 'walking', 'along', 'a', 'raised', 'balance', 'beam', '.']
전체 소스 토큰: ['<sos>', 'ein', 'mädchen', 'in', 'einem', 'jeanskleid', 'läuft', 'über', 'einen', 'erhöhten', 'schwebebalken', '.', '<eos>']
소스 문장 인덱스: [2, 5, 25, 7, 6, 0, 83, 42, 19, 3199, 2669, 4, 3]
모델 출력 결과: a girl in a <unk> outfit runs across a <unk> course . <eos>


In [34]:
src = tokenize_de("Guten Tag.")

print(f'소스 문장: {src}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['Guten', 'Tag', '.']
전체 소스 토큰: ['<sos>', 'guten', 'tag', '.', '<eos>']
소스 문장 인덱스: [2, 3799, 200, 4, 3]
모델 출력 결과: a day at night . <eos>


In [35]:
src = tokenize_de("Guten Morgen.")

print(f'소스 문장: {src}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['Guten', 'Morgen', '.']
전체 소스 토큰: ['<sos>', 'guten', 'morgen', '.', '<eos>']
소스 문장 인덱스: [2, 3799, 3352, 4, 3]
모델 출력 결과: protesters are going to start . <eos>
